In [1]:
%pip install xgboost
%pip install seaborn
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


In [3]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [17]:
import lightgbm as lgb

In [5]:
train_df = pd.read_csv(r'E:\home_credit_assignment\data\processed\train_aggregated.csv')
test_df = pd.read_csv(r'E:\home_credit_assignment\data\processed\test_aggregated.csv') 

In [ ]:
# Train test Split
X = train_df.drop(columns=['TARGET'])
y = train_df['TARGET']

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
# Identify low imp;ortance features
importance_df = pd.read_csv('../data/processed/feature_importance.csv')
low_importance_features = importance_df[importance_df['importance'] == 0]['feature'].tolist()
print(f"Removing {len(low_importance_features)} non-important features.")


Removing 89 non-important features.


In [8]:
# Removing Low Importance Features

X_train = X_train.drop(columns=[col for col in low_importance_features if col in X_train.columns])
X_val = X_val.drop(columns=[col for col in low_importance_features if col in X_val.columns])
test_df = test_df.drop(columns=[col for col in low_importance_features if col in test_df.columns])


In [ ]:
# Removing High colinearity Columns

corr_matrix = X_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
high_corr_features = [column for column in upper.columns if any(upper[column] > 0.98)]
print(f"Removing {len(high_corr_features)} highly collinear features.")


In [ ]:
X_train = X_train.drop(columns=high_corr_features)
X_val = X_val.drop(columns=high_corr_features)
test_df = test_df.drop(columns=high_corr_features)


In [9]:
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"test_df shape: {test_df.shape}")


X_train shape: (246008, 513)
X_val shape: (61503, 513)
test_df shape: (48744, 513)


In [ ]:
# Remove Inf Values
X_train = X_train.replace([np.inf, -np.inf], 0)
X_val = X_val.replace([np.inf, -np.inf], 0)
test_df = test_df.replace([np.inf, -np.inf], 0)


In [ ]:
X_train.to_csv('../data/processed/X_train_pruned.csv', index=False)
X_val.to_csv('../data/processed/X_val_pruned.csv', index=False)
test_df.to_csv('../data/processed/test_df_pruned.csv', index=False)


XGBoost Training

In [21]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(test_df)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum(),
    'seed': 42
}

evals = [(dtrain, 'train'), (dval, 'validation')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=evals,
    early_stopping_rounds=50,
    verbose_eval=100
)


[0]	train-auc:0.70081	validation-auc:0.69056
[100]	train-auc:0.81623	validation-auc:0.77511
[200]	train-auc:0.84515	validation-auc:0.78109
[300]	train-auc:0.86590	validation-auc:0.78314
[400]	train-auc:0.88270	validation-auc:0.78358
[467]	train-auc:0.89228	validation-auc:0.78324


LGBoost Training

In [22]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbosity': -1,
    'seed': 42
}

lgb_model = lgb.train(
    lgb_params,
    lgb_train,
    valid_sets=[lgb_train, lgb_val],
    num_boost_round=1000,
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),lgb.log_evaluation(period=100)
    ]
)

lgb_preds = lgb_model.predict(test_df, num_iteration=lgb_model.best_iteration)


Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.800571	valid_1's auc: 0.77649
[200]	training's auc: 0.828445	valid_1's auc: 0.782279
[300]	training's auc: 0.85028	valid_1's auc: 0.783583
Early stopping, best iteration is:
[343]	training's auc: 0.858684	valid_1's auc: 0.783967


Ensembleling LGB and XGB

In [23]:
dval = xgb.DMatrix(X_val)
xgb_val_preds = xgb_model.predict(dval)

# LightGBM predictions
lgb_val_preds = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)
# Ensemble
#ensemble_val_preds = (xgb_val_preds + lgb_val_preds) / 2
ensemble_val_preds = (0.2 * xgb_val_preds) + (0.8 * lgb_val_preds)
# Evaluate AUC
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(y_val, ensemble_val_preds)
print(f"Ensemble Validation AUC: {auc_score:.4f}")


Ensemble Validation AUC: 0.7865


In [12]:
%pip install catboost 

Note: you may need to restart the kernel to use updated packages.


CATBoost Training

In [24]:
from catboost import CatBoostClassifier

model_cb = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    eval_metric='AUC',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=50
)

model_cb.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)


0:	test: 0.6305987	best: 0.6305987 (0)	total: 97.6ms	remaining: 1m 37s
100:	test: 0.7703775	best: 0.7703775 (100)	total: 9.92s	remaining: 1m 28s
200:	test: 0.7781843	best: 0.7781843 (200)	total: 24.7s	remaining: 1m 38s
300:	test: 0.7819735	best: 0.7819735 (300)	total: 39.1s	remaining: 1m 30s
400:	test: 0.7835297	best: 0.7835311 (398)	total: 53.2s	remaining: 1m 19s
500:	test: 0.7844103	best: 0.7844119 (499)	total: 1m 7s	remaining: 1m 6s
600:	test: 0.7852751	best: 0.7852820 (598)	total: 1m 21s	remaining: 53.8s
700:	test: 0.7860169	best: 0.7860169 (700)	total: 1m 35s	remaining: 40.6s
800:	test: 0.7864774	best: 0.7864774 (800)	total: 1m 49s	remaining: 27.1s
900:	test: 0.7870507	best: 0.7870507 (900)	total: 2m 3s	remaining: 13.5s
999:	test: 0.7872595	best: 0.7872836 (996)	total: 2m 17s	remaining: 0us

bestTest = 0.7872835769
bestIteration = 996

Shrink model to first 997 iterations.


Stratified K-Fold Training

In [13]:
from sklearn.model_selection import StratifiedKFold
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

auc_scores_xgb = []
auc_scores_lgb = []
auc_scores_cb = []


In [14]:
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

auc_scores_xgb = []
auc_scores_lgb = []
auc_scores_cb = []


In [15]:
oof_preds_xgb = np.zeros(X.shape[0])
oof_preds_lgb = np.zeros(X.shape[0])
oof_preds_cb = np.zeros(X.shape[0])


In [18]:
from catboost import CatBoostClassifier
for fold,(train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
    print(f"\n--- Fold {fold + 1} ---")
    X_tr, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # XGBoost
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dval = xgb.DMatrix(X_val_fold, label=y_val_fold)

    xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'learning_rate': 0.05,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'seed': 42
    }

    model_xgb = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=1000,
        evals=[(dval, 'validation')],
        early_stopping_rounds=50,
        verbose_eval=100
    )

    preds_xgb = model_xgb.predict(dval)
    auc_xgb = roc_auc_score(y_val_fold, preds_xgb)
    auc_scores_xgb.append(auc_xgb)
    oof_preds_xgb[val_idx] = preds_xgb

    print(f"XGBoost Fold AUC: {auc_xgb:.4f}")

    # LightGBM
    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_val = lgb.Dataset(X_val_fold, y_val_fold, reference=lgb_train)

    lgb_params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbosity': -1,
        'seed': 42
    }

    model_lgb = lgb.train(
        lgb_params,
        lgb_train,
        num_boost_round=1000,
        valid_sets=[lgb_val],
        callbacks=[
        lgb.early_stopping(stopping_rounds=50),lgb.log_evaluation(period=100)
    ]
    )

    preds_lgb = model_lgb.predict(X_val_fold, num_iteration=model_lgb.best_iteration)
    auc_lgb = roc_auc_score(y_val_fold, preds_lgb)
    auc_scores_lgb.append(auc_lgb)
    oof_preds_lgb[val_idx] = preds_lgb

    print(f"LightGBM Fold AUC: {auc_lgb:.4f}")

    # CatBoost
    model_cb = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        eval_metric='AUC',
        random_seed=42,
        verbose=100,
        early_stopping_rounds=50
    )

    model_cb.fit(X_tr, y_tr, eval_set=(X_val_fold, y_val_fold), use_best_model=True)
    preds_cb = model_cb.predict_proba(X_val_fold)[:, 1]
    auc_cb = roc_auc_score(y_val_fold, preds_cb)
    auc_scores_cb.append(auc_cb)
    oof_preds_cb[val_idx] = preds_cb

    print(f"CatBoost Fold AUC: {auc_cb:.4f}")



--- Fold 1 ---
[0]	validation-auc:0.68041
[100]	validation-auc:0.76907
[200]	validation-auc:0.77505
[300]	validation-auc:0.77674
[400]	validation-auc:0.77661
[500]	validation-auc:0.77705
[546]	validation-auc:0.77667
XGBoost Fold AUC: 0.7767
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.769143
[200]	valid_0's auc: 0.774634
[300]	valid_0's auc: 0.775919
Early stopping, best iteration is:
[283]	valid_0's auc: 0.776189
LightGBM Fold AUC: 0.7762
0:	test: 0.6060345	best: 0.6060345 (0)	total: 309ms	remaining: 5m 8s
100:	test: 0.7608528	best: 0.7608528 (100)	total: 13.5s	remaining: 2m
200:	test: 0.7694129	best: 0.7694129 (200)	total: 35.2s	remaining: 2m 19s
300:	test: 0.7733826	best: 0.7733837 (299)	total: 56.6s	remaining: 2m 11s
400:	test: 0.7755066	best: 0.7755333 (397)	total: 1m 11s	remaining: 1m 46s
500:	test: 0.7768035	best: 0.7768035 (500)	total: 1m 21s	remaining: 1m 21s
600:	test: 0.7775926	best: 0.7775926 (600)	total: 1m 32s	remaining: 1m 1s
700:	

EVALUATION

In [19]:
print("Cross-Validation Results:")
print(f"XGBoost CV AUC: {np.mean(auc_scores_xgb):.4f} ± {np.std(auc_scores_xgb):.4f}")
print(f"LightGBM CV AUC: {np.mean(auc_scores_lgb):.4f} ± {np.std(auc_scores_lgb):.4f}")
print(f"CatBoost CV AUC: {np.mean(auc_scores_cb):.4f} ± {np.std(auc_scores_cb):.4f}")


Cross-Validation Results:
XGBoost CV AUC: 0.7810 ± 0.0034
LightGBM CV AUC: 0.7801 ± 0.0022
CatBoost CV AUC: 0.7836 ± 0.0021


In [25]:
dval = xgb.DMatrix(X_val, label=y_val)
xgb_pred_val = xgb_model.predict(dval)

lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)
lgb_val_preds = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)

cb_val_pred = model_cb.predict_proba(X_val)[:, 1]

In [26]:
ensemble_preds_val = (xgb_pred_val + lgb_val_preds + cb_val_pred) / 3

from sklearn.metrics import roc_auc_score

ensemble_auc = roc_auc_score(y_val, ensemble_preds_val)
print(f" Simple Ensemble Validation AUC: {ensemble_auc:.4f}")


 Simple Ensemble Validation AUC: 0.7873


In [27]:
dtest = xgb.DMatrix(test_df)
xgb_test = xgb_model.predict(dtest)

lgb_test = lgb_model.predict(test_df, num_iteration=lgb_model.best_iteration)

cb_test = model_cb.predict_proba(test_df)[:, 1]

In [28]:
ensemble_preds_test = (xgb_test + lgb_test + cb_test) / 3

In [29]:
raw_test = pd.read_csv('../data/raw/application_test.csv')
test_ids = raw_test['SK_ID_CURR']

In [30]:
submission = pd.DataFrame({
    'SK_ID_CURR': test_ids,
    'TARGET': ensemble_preds_test
})

submission.to_csv('../outputs/submission_simple_ensemble.csv', index=False)
